# Xarray-spatial
### User Guide: Zonal Crosstab
-----

Xarray-spatial's `zonal_crosstab` function calculates cross-tabulated statistics between two raster datasets, helping identify patterns and relationships in spatial data.

In this notebook, we'll analyze how **land surface temperature varies with vegetation density** using Landsat 8 satellite imagery. We'll use:
- **NDVI** (Normalized Difference Vegetation Index) calculated from NIR and Red bands to measure vegetation density
- **Thermal infrared** (TIR) band to measure land surface temperature

This demonstrates a common remote sensing analysis: understanding the relationship between vegetation cover and urban heat islands.

**Contents:**
- [Load Landsat Data](#Load-Landsat-Data)
- [Calculate NDVI and Temperature](#Calculate-NDVI-and-Temperature)
- [2D Zonal Crosstab](#2D-Zonal-Crosstab): Temperature distribution by vegetation class

-----------

## Load Landsat Data

We'll use Landsat 8 imagery which includes multiple spectral bands. To download the example data, run `xrspatial fetch-data` from the repository root directory.

The key bands we need are:
- **Band 4 (Red)**: Used with NIR to calculate NDVI
- **Band 5 (NIR)**: Near-infrared, reflected strongly by vegetation
- **Band 10 (TIR1)**: Thermal infrared for surface temperature

In [ ]:
import numpy as np
import rioxarray
import matplotlib.pyplot as plt

from datashader.transfer_functions import shade, stack, Images

from xrspatial.multispectral import ndvi
from xrspatial.classify import quantile
from xrspatial import zonal_crosstab
from xrspatial.zonal import stats as zonal_stats

#### Load the spectral bands

We load the Red, NIR, and Thermal bands from Landsat 8 imagery.

In [ ]:
SCENE_ID = "LC80030172015001LGN00"
DATA_PATH = "../../xrspatial-examples/data"

# Load NIR (Band 5), Red (Band 4), and Thermal (Band 10)
nir = rioxarray.open_rasterio(f"{DATA_PATH}/{SCENE_ID}_B5.tiff").load()[0]
red = rioxarray.open_rasterio(f"{DATA_PATH}/{SCENE_ID}_B4.tiff").load()[0]
thermal = rioxarray.open_rasterio(f"{DATA_PATH}/{SCENE_ID}_B10.tiff").load()[0]

print(f"Image dimensions: {nir.shape}")
print(f"NIR band range: {float(nir.min()):.0f} - {float(nir.max()):.0f}")
print(f"Thermal band range: {float(thermal.min()):.0f} - {float(thermal.max()):.0f}")

## Calculate NDVI and Temperature

### NDVI (Vegetation Index)

NDVI measures vegetation density using the formula: `(NIR - Red) / (NIR + Red)`. Values range from -1 to 1, where higher values indicate denser vegetation.

In [ ]:
# Calculate NDVI
ndvi_agg = ndvi(nir_agg=nir, red_agg=red)
ndvi_agg.name = "NDVI"

# Visualize NDVI
fig, ax = plt.subplots(figsize=(10, 8))
ndvi_agg.plot.imshow(ax=ax, cmap="RdYlGn", vmin=-0.5, vmax=0.8)
ax.set_title("NDVI - Vegetation Density")
plt.tight_layout()

### Land Surface Temperature

The thermal band (B10) provides brightness temperature values. For this analysis, we'll use the raw digital numbers as a relative temperature proxy.

In [ ]:
# Visualize thermal band
fig, ax = plt.subplots(figsize=(10, 8))
thermal.plot.imshow(ax=ax, cmap="magma")
ax.set_title("Thermal Band (B10) - Land Surface Temperature Proxy")
plt.tight_layout()

### Visualize Both Layers Together

In [ ]:
ndvi_img = shade(ndvi_agg, cmap=plt.get_cmap("RdYlGn"), how="linear")
ndvi_img.name = "NDVI"

thermal_img = shade(thermal, cmap=plt.get_cmap("magma"), how="linear")
thermal_img.name = "Temperature"

imgs = Images(ndvi_img, thermal_img)
imgs.num_cols = 2
imgs

## 2D Zonal Crosstab

The `zonal_crosstab` function calculates cross-tabulated statistics between a **zones** raster and a **values** raster.

We'll:
1. Classify NDVI into vegetation zones (from bare/water to dense vegetation)
2. Classify thermal data into temperature classes
3. Cross-tabulate to see how temperature is distributed across vegetation zones

In [ ]:
# Create vegetation zones from NDVI
n_veg_classes = 5
ndvi_zones = quantile(ndvi_agg, k=n_veg_classes, name='Vegetation Zones')

# Classify thermal data
n_temp_classes = 5
temp_classes = quantile(thermal, k=n_temp_classes, name='Temperature Classes')

print(f"Created {n_veg_classes} vegetation zones and {n_temp_classes} temperature classes")

### Visualize the Classified Data

In [ ]:
shaded_veg_zones = shade(ndvi_zones, cmap=plt.get_cmap("RdYlGn"), how="linear")
shaded_veg_zones.name = "Vegetation Zones"

shaded_temp_classes = shade(temp_classes, cmap=plt.get_cmap("magma"), how="linear")
shaded_temp_classes.name = "Temperature Classes"

imgs = Images(shaded_veg_zones, shaded_temp_classes)
imgs.num_cols = 2
imgs

### Helper Function for Zone Labels

This utility function extracts the value range for each classified zone.

In [ ]:
def bin_ranges(classified_data, original_data, unit="", decimals=2):
    """Calculate the value range for each bin/class."""
    bins = np.unique(classified_data.data[~np.isnan(classified_data.data)])
    ranges = []
    for b in bins:
        bin_data = original_data.data[classified_data.data == b]
        min_val = np.nanmin(bin_data)
        max_val = np.nanmax(bin_data)
        ranges.append(f'{min_val:.{decimals}f}{unit} - {max_val:.{decimals}f}{unit}')
    return ranges

In [ ]:
# Get human-readable labels for zones
veg_labels = bin_ranges(ndvi_zones, ndvi_agg, decimals=2)
temp_labels = bin_ranges(temp_classes, thermal, decimals=0)

print("Vegetation zones (NDVI ranges):")
for i, label in enumerate(veg_labels, 1):
    print(f"  Zone {i}: {label}")

In [ ]:
print("Temperature classes (thermal DN ranges):")
for i, label in enumerate(temp_labels, 1):
    print(f"  Class {i}: {label}")

### Run Zonal Crosstab

Now we calculate the cross-tabulation to see how temperature classes are distributed across vegetation zones.

In [ ]:
# Calculate cross-tabulation with percentage aggregation
crosstab_result = zonal_crosstab(ndvi_zones, temp_classes, agg='percentage')

# Add readable labels
crosstab_result['zone'] = veg_labels
crosstab_result.columns = ['Vegetation Zone', *temp_labels]
crosstab_result.set_index('Vegetation Zone', inplace=True)

crosstab_result

### Interpretation

Each cell shows the **percentage** of pixels in a vegetation zone that fall into each temperature class.

- **Low NDVI** (bare soil, water, urban): Higher percentages in warmer temperature classes
- **High NDVI** (dense vegetation): Higher percentages in cooler temperature classes

This demonstrates the well-known **urban heat island effect** - vegetated areas tend to be cooler than bare or developed areas.

In [ ]:
# Visualize as stacked bar chart
ax = crosstab_result.plot(kind="bar", stacked=True, figsize=(12, 6), colormap="magma")
ax.set_xlabel("Vegetation Zone (NDVI)")
ax.set_ylabel("Percentage")
ax.set_title("Temperature Distribution by Vegetation Zone")
ax.legend(title="Temperature Class", bbox_to_anchor=(1.02, 1), loc='upper left')
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

## Zonal Statistics

For calculating summary statistics like mean, min, max, std within each zone, use `xrspatial.zonal.stats` instead of `zonal_crosstab`.

Let's calculate the mean temperature within each vegetation zone.

In [ ]:
# Calculate zonal statistics for thermal band within each vegetation zone
mean_temp_by_veg = zonal_stats(ndvi_zones, thermal, stats_funcs=['mean'])
mean_temp_by_veg['Vegetation Zone'] = veg_labels
mean_temp_by_veg = mean_temp_by_veg[['Vegetation Zone', 'mean']].rename(columns={'mean': 'Mean Temperature (DN)'})
mean_temp_by_veg.set_index('Vegetation Zone', inplace=True)

mean_temp_by_veg

In [ ]:
# Visualize mean temperature by vegetation zone
ax = mean_temp_by_veg.plot(kind="bar", figsize=(10, 5), legend=False, color="orangered")
ax.set_xlabel("Vegetation Zone (NDVI)")
ax.set_ylabel("Mean Temperature (DN)")
ax.set_title("Mean Land Surface Temperature by Vegetation Density")
plt.xticks(rotation=45, ha='right')
plt.tight_layout()

## Summary

In this notebook, we demonstrated how to use `zonal_crosstab` to analyze the relationship between vegetation density (NDVI) and land surface temperature using Landsat 8 imagery.

**Key findings:**
- Areas with higher NDVI (more vegetation) tend to have lower surface temperatures
- This pattern is consistent with the urban heat island effect, where vegetation provides cooling through evapotranspiration

**Use cases for `zonal_crosstab`:**
- Land use / land cover analysis
- Environmental monitoring
- Urban planning and heat island studies
- Agricultural analysis